Возьмём текст "Критика чистого разума" Иммануила Канта (text.txt) и применим к нему build_exceptions, строющую исключения их стандартных правил капитализации.

In [1]:
import re
import sys

def build_exceptions(text):
    exceptions = [0] * len(text)
    
    # правило 1: первая буква текста
    for i, c in enumerate(text):
        if c.isalpha():
            if c.isupper():
                exceptions[i] = 0
            break

    # правило 2: после [.?!] и пробела/переноса строки
    for match in re.finditer(r'[\.\!\?][\s\n]+([A-Za-z])', text):
        idx = match.start(1)
        if text[idx].isupper():
            exceptions[idx] = 0 

    # правило 3: одиночная 'I'
    for match in re.finditer(r'[^A-Za-z]I[^A-Za-z]', text):
        idx = match.start() + 1
        if text[idx] == 'I':
            exceptions[idx] = 0

    # все остальные заглавные — исключения
    for i, c in enumerate(text):
        if c.isupper() and exceptions[i] == 0:
            # если это не по правилу, ставим 1
            is_exception = True
            # (уже учтено? если да — пропускаем)
            if (
                i == 0 or
                (i > 1 and text[i-2:i] in {'. ', '! ', '? ', '.\n', '!\n', '?\n'}) or
                (i > 0 and text[i-1] == ' ' and i+1 < len(text) and text[i+1] == ' ')
            ):
                is_exception = False
            if is_exception:
                exceptions[i] = 1

    return exceptions

def decapitalize(text):
    return ''.join(c.lower() if c.isupper() else c for c in text)


filename = 'text.txt'
with open(filename, 'r', encoding='ASCII') as f:
    text = f.read()

exceptions = build_exceptions(text)
lower_text = decapitalize(text)

# Сохраняем результат
with open('decapitalized.txt', 'w', encoding='ASCII') as f:
    f.write(lower_text)

with open('exceptions.txt', 'w', encoding='ASCII') as f:
    f.write(''.join((str(j) for j in exceptions)))

Используем лишь стандартные правила капитализации.  
Как выяснится дальше, число заглавных букв и так невелико, а число полученных исключений = 1/260 от длины всего текста

In [1]:
filename = 'text.txt'
with open(filename, 'r', encoding='ASCII') as f:
    text = f.read()

In [5]:
print(f'Символов в тексте: {len(text)}')

uppercase_count = sum(1 for char in text if char.isupper())
print(f"Количество заглавных букв в файле: {uppercase_count}")

Символов в тексте: 1268316
Количество заглавных букв в файле: 10131


In [15]:
filename = 'exceptions.txt'
with open(filename, 'r', encoding='ASCII') as f:
    exceptions_text = f.read()

In [ ]:
print(len(exceptions))
print(f'Всего исключений: {sum(exceptions)}')
percent_1 = round(sum(exceptions) / len(exceptions), 5)
print(f'Доля исключений из всего текста: {percent_1 * 100}% (1/260)')

print(f'Число объяснённых заглавных букв стандартными правилами капитализации = {uppercase_count - sum(exceptions)}')

1268316
Всего исключений: 4835
Доля исключений из всего текста: 0.381% (1/260)
Число объяснённых заглавных букв стандартными правилами капитализации = 5296


Почти половина всех заглавных букв была объяснена стандартными правилами капитализации.  
В тексте можно найти ещё пару правил, например, часто встречаются комментарии вида:
текст*  
   
*Комментарий [Продолжение комментария "Цитата"]

Но большое количество заглавных букв получается из-за того, что часто названия глав и разделов написаны капсом


In [94]:
import math
entropy = -(math.log2(percent_1)*percent_1 + math.log2(1-percent_1)*(1-percent_1))
print(f'Энтропия = {round(entropy, 10)} \n')

print(f'Если передать позиции 4835 исключений, зашифровав в 21 бит каждое, то получится {4835*21} бит или {round(4835*21 / 8 / 1024, 2)} кб')
print(f'Тогда степень сжатия = {round(4835*21 / len(exceptions), 5)} => baseline = 8% сжатия битовой маски')

Энтропия = 0.0361033181 

Если передать позиции 4835 исключений, зашифровав в 21 бит каждое, то получится 101535 бит или 12.39 кб
Тогда степень сжатия = 0.08005 => baseline = 8% сжатия битовой маски


# Арифметическое кодирование


Алгоритм взят отсюда: https://github.com/tommyod/arithmetic-coding 

In [98]:
from arithmetic_coding import ArithmeticEncoder


message = [str(i) for i in exceptions] + ['<EOM>']
frequencies = {'0': len(exceptions) - sum(exceptions), '1': 4835, '<EOM>':1}
encoder = ArithmeticEncoder(frequencies=frequencies, bits=128)
bits = list(encoder.encode(message))
bits[:10]

[1, 1, 0, 0, 0, 0, 1, 1, 1, 1]

In [97]:
print(f'Длина получившейся битовой строки = {len(bits)} бит или {round(len(bits) / 8 / 1024, 3)} кб')
print(f'Что составляет {round(len(bits) / len(exceptions), 7)} от длины всей маски')
print(f'Получившийся результат лишь на 5-ом знаке отличается от энтропии')
print(f'Полученный результат уже достаточно хорош, чтобы мы могли выбрать арифметическое кодирование')

Длина получившейся битовой строки = 45835 бит или 5.595 кб
Что составляет 0.0361385 от длины всей маски
Получившийся результат лишь на 5-ом знаке отличается от энтропии
Полученный результат уже достаточно хорош, чтобы мы могли выбрать арифметическое кодирование
